In [ ]:
%pip install -r requirements.txt

In [1]:
#use logger
import logging
import time
import utils.config as config
import utils.io_utils as io_utils
if __name__ == "__main__":
    try:
        #logging.basicConfig(level=logging.DEBUG)
        start_time = time.time()
        io_utils.extract_EaglePack_and_process(config.input_path)
        end_time = time.time()
        print(f"Total processing time: {end_time - start_time:.2f} seconds")
    finally:
        if config.use_LLM:
            config.llm_processor.stop()
    print("Processing completed")

Not using LLM


Images Queued: 100%|██████████| 107/107 [00:00<00:00, 113.97it/s]


Total processing time: 125.95 seconds
Processing completed
